In [1]:
from numpy.random import rand
import numpy as np
from numpy.random import choice
from numpy import asarray
from numpy import clip
from numpy import argmin
from numpy import min
from numpy import around
from matplotlib import pyplot
import random
from numpy.random import choice
import copy

In [2]:
# number of fragments
m = 20
# number of sites
n = 4
#population size
pop_size = 200
# number of transaction
l = 20

sol = np.random.randint(0,n,size=(pop_size,m))
sol2 = copy.deepcopy(sol)

frequency = np.random.randint(0,10,size=(n,l)) # frequency of transaction l on site s

trfr = np.random.randint(0,10,size=(l,m)) # Transaction Fragment Dependency

q = np.random.randint(0,10,size=(l,m,m))

# Indirect transaction-fragment dependencies considering frequencies
# amount of data transfer from site of a fragment to site of another fragment i.e. qfr
qfr = np.zeros(shape=(l,m,m))
for i in range(l):
    for j in range(m):
        for k in range(m):
            qfr[i,j,k] = q[i,j,k] * np.sum(frequency[:,i])

# Inter fragment dependencies i.e. frdep
# Amount of data transmission from site of fj1 to site of fj2 considering indirect dependencies
frdep = np.zeros(shape=(m,m))
for i in range(m):
    for j in range(m):
        frdep[i,j] = np.sum(qfr[:,i,j])
        

        
# Site-fragment dependencies i.e. stfr
# Amount of data from fj accessed by si in unit time
stfr = np.dot(frequency,trfr)

# unit data transmission cost between two sites i.e. UC
uc = np.random.randint(0,5,size=(n,n))
uc = (uc + uc.T)/2




In [3]:
def calculation(solution):
    cost1 = 0 # partial cost
    cost2 = 0 # direct cost
    cost = 0
    for j1 in range(m):
        for j2 in range(m):
            cost1 = cost1 + (frdep[j1,j2] + uc[solution[j1],solution[j2]])
    for j in range(m):        
        for q in range(n):   # partialco
            r1 = 0
            for k in range(l):
                r1 = r1 + (frequency[solution[j],k] * trfr[k,j])
                cost2 = cost2 + (uc[solution[j],q] * r1)
    cost = cost1 + cost2
    return cost
    
def crossover(x,y,cr = 0.8):
    R = random.randrange(0,1)
    if R < cr:
        cutp1 = random.randrange(1,m)
        cutp2 = random.randrange(1,m)
        for i in range(cutp1):
            x[i],y[i] = y[i], x[i]
        for i in range(cutp2,m):
            x[i],y[i] = y[i], x[i]
    
    return x,y

# def crossover(x,y,cr = 0.8):
#     x1 = copy.deepcopy(x)
#     y1= copy.deepcopy(y)
#     R = random.randrange(0,1)
#     if R < cr:
#         cutp1 = random.randrange(1,m)
#         cutp2 = random.randrange(1,m)
#         for i in range(cutp1):
#             x[i],y[i] = y[i], x[i]
#         for i in range(cutp2,m):
#             x[i],y[i] = y[i], x[i]
    
#     if calculation(x) < calculation(x1):
#         p = x
#     else:
#         p = x1
#     if calculation(y) < calculation(y1):
#         q = y
#     else:
#         q = y1
#     return p,q


def mutation(X,Y):
    if calculation(X) < calculation(Y):
            Z = X
    elif calculation(Y) < calculation(X):
        Z = Y
    else:
        rand = np.random.randint(0,2)
        if rand == 0:
            Z = X
        else:
            Z = Y
    return Z

def VNS(X):
    # neighbour structure will be a matrix of dimension 1Xm. where m = no of fragments = 20
    for k in range(10):
        Y = X
        for p in range(5):
            for q in range(3): # changing 3 sites randomly
                fragment = np.random.randint(0,20)
                site = np.random.randint(0,n)
                Z = Y
                Z[fragment] = site
            if calculation(Z) < calculation(Y):
                Y = Z
        if calculation(Y) < calculation(X):
            X = Y
    return X
                
        
        

In [4]:
COST = [calculation(solution) for solution in sol]
newcost = [(max(COST)-COST[i]) for i in range(pop_size)]
for i in range(1): # number of iteration
    for j in range(20): # for all solution in sol
        candidates = [candidate for candidate in range(pop_size) if candidate != j]
        R1, R2, R3 = sol[choice(candidates, 3, replace=False)]
        A1,B1 = crossover(R2,R3)
        C = mutation(A1,B1)
        A2,B2 = crossover(R1,C)
        C = mutation(A2,B2)
        A3,B3 = crossover(sol[j],C)
        C = mutation(A3,B3)
        D = VNS(C)
        if calculation(D) < calculation(sol[j]):
            sol[j] = D
        
    
        

In [5]:
final_cost = [calculation(i) for i in sol]
print("final solution :", sol[final_cost.index(min(final_cost))] )
print("cost of solution : ",min(final_cost))

final solution : [1 1 0 1 0 1 2 1 2 0 1 0 1 0 0 0 0 0 0 2]
cost of solution :  1543578.5


In [6]:
final_cost = [calculation(i) for i in sol]
print("final solution :", sol[final_cost.index(min(final_cost))] )
print("cost of solution : ",min(final_cost))

final solution : [1 1 0 1 0 1 2 1 2 0 1 0 1 0 0 0 0 0 0 2]
cost of solution :  1543578.5


In [7]:
sol2 = np.array(sol2)
initial_cost = [calculation(i) for i in sol2]
print("minimum cost solution in initial population :", sol2[initial_cost.index(min(initial_cost))])
print("cost of solution",min(initial_cost))

minimum cost solution in initial population : [2 0 1 1 2 2 0 1 2 1 1 2 0 0 2 3 2 2 1 1]
cost of solution 1524028.0
